In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-g2i6oi9s
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-g2i6oi9s
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=a3b2a82db319c78bcb63c81f3e633e3f235304d1e5ce62255c78b03fa08907c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-f1irclyj/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
!nvidia-smi

Fri Apr 10 16:30:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
!ls

sample_data  src


In [26]:
%%cuda --name helloCUDA.cu
#include <iostream>
#include <random>
#include <algorithm>
#define LOOP(x) for(int t##x = 0; t##x < x; t##x++)
#define cudaSafeCall(call)  \
        do {\
            cudaError_t err = call;\
            if (cudaSuccess != err) \
            {\
                std::cerr << "CUDA error in " << __FILE__ << "(" << __LINE__ << "): " \
                    << cudaGetErrorString(err);\
                exit(EXIT_FAILURE);\
            }\
        } while(0)

using namespace std;

void gpu_error(cudaError_t const &code) {
    if(code != cudaSuccess)
    {
        cerr << "GPUError: Code " << code << " : " << cudaGetErrorString(code) << endl;
        exit( EXIT_FAILURE );
    }
}

__global__ void precompute(int och, int ch, float* kernel_weights, float *U)
{
    int x = threadIdx.x;
    int bid = blockIdx.x;
    int offset = bid*ch + x;
    // int m = 2, n = 3;
    
    float g[4][3] = {
        {1, 0, 0},
        {0.5, 0.5, 0.5},
        {0.5, -0.5, 0.5},
        {0, 0, 1}
    };
    
    float g_t[3][4] ={
        {1, 0.5, 0.5, 0},
        {0, 0.5, -0.5, 0},
        {0, 0.5, 0.5, 1}
    };
    float *temp = (float *)malloc(3*4*sizeof(float));
    for(int i = 0; i <3; ++i)
    {
        for(int j = 0; j <4; ++j)
        {
            temp[i*4+j] = 0;
            for(int k = 0; k <3; ++k)
            {
                temp[i*4+j] += kernel_weights[offset*3*3+i*3+k] * g_t[k][j];
            }
        }
    }
    for(int i = 0; i <4; ++i)
    {
        for(int j = 0; j <4; ++j)
        {
            U[offset*4*4+i*4+j] = 0;
            for(int k = 0; k <3; ++k)
            {
                U[offset*4*4+i*4+j] += g[i][k] * temp[k*4+j];
            }
        }
    }
}
__global__ void uv(int tch, int ch, float *devfin, float *U,  float *V)
{
    int x = threadIdx.x;
    for(int i = 0; i <4; ++i)
    {
        for(int j = 0; j <4; ++j)
        {
            devfin[x*4*4 + i*4 + j] = U[((x*ch+tch)*4+i)*4+j]*V[i*4+j];
            //devfin[offset2*4*4 + i*4 + j] = 3.0;//U[offset*4*4+i*4+j]*V[i*4+j];
            
        }
    }
}
__global__ void amul(int tbs, int tp, int tq, int bs, int och, int p, int q, float *devsum, float *devY)
{
    float A_t[2][4] = {
        {1, 1, 1, 0},
        {0, 1, -1,-1}
    };
    float A[4][2] = {
        {1,0},
        {1,1},
        {1,-1},
        {0,1}
    };
    int x = threadIdx.x;
    float *temp = (float *)malloc(2*4*sizeof(float));
    for(int i = 0; i <2; ++i)
    {
        for(int j = 0; j <4; ++j)
        {
            temp[i*4+j] = 0;
            for(int k = 0; k <4; ++k)
            {
                temp[i*4+j] += A_t[i][k] * devsum[((((tbs*och+x)*p+tp)*q+tq)*4+k)*4+j];
            }
        }
    }
    for(int i = 0; i <2; ++i)
    {
        for(int j = 0; j <2; ++j)
        {
            devY[((((tbs*och+x)*p+tp)*q+tq)*2+i)*2+j] = 0;
            for(int k = 0; k <4; ++k)
            {
                devY[((((tbs*och+x)*p+tp)*q+tq)*2+i)*2+j] += temp[i*4+k] * A[k][j];
            }
        }
    }
}

__global__ void paddev(float *devin, float *devinnopad, int h, int w, int pad)
{
    int newh = gridDim.y;
    int neww = gridDim.z;
    int tbsch = blockIdx.x;
    int tnewh = blockIdx.y;
    int tneww = blockIdx.z;
    int newhw = newh*neww;
    int hw = h*w;
    int th = tnewh-pad;
    int tw = tneww-pad;
    
    if(th >= 0 && th < h && tw >= 0 && tw < w)
    {
        devin[tbsch*newhw + tnewh*neww + tneww] = devinnopad[tbsch*hw + th*w + tw];
    }
    else
    {
        devin[tbsch*newhw + tnewh*neww + tneww] = 0;
    }
    
}
    
__global__ void tile(int bs, int p, int q, int ch, float *devin, float *devout, float *devsum, float *devY, float *devU, int h, int w, int och, float *&devfin)
{
    float thrtile[4][4];
    /*
    int bs, p, q, ch;
    bs = gridDim.x;
    p = gridDim.y;
    q = gridDim.z;
    ch = blockDim.x;*/
    
    int tbs, tp, tq, tch;
    tbs = blockIdx.x;
    tp = blockIdx.y;
    tq = blockIdx.z;
    tch = threadIdx.x;
    // copy the tiles to thrtile
    int offset1 = (tbs*ch + tch)*h*w;
    // float *t = thrtile;
    for(int th = 2*tp, i = 0; i < 4; th++, i++)
    {
        for(int tw = 2*tq, j = 0; j < 4; tw++, j++)
        {
            thrtile[i][j] = devin[offset1 + th*w + tw];
        }
    }
    float B[4][4] = {
        {1,0,0,0},
        {0,1,-1,1},
        {-1,1,1,0},
        {0,0,0,-1}
    };
    float B_t[4][4] = {
        {1,0,-1,0},
        {0,1,1,0},
        {0,-1,1,0},
        {0,1,0,-1}
    };
    //Calculation of V
    float temp[4][4];
    float *V = (float *)  malloc(16*sizeof(float));
    for(int i = 0; i <4; ++i)
    {
        for(int j = 0; j <4; ++j)
        {
            temp[i][j] = 0;
            for(int k = 0; k <4; ++k)
            {
                temp[i][j] += thrtile[i][k] * B[k][j];
            }
     
        }
    }
    for(int i = 0; i <4; ++i)
    {
        for(int j = 0; j <4; ++j)
        {
            V[i*4+j] = 0;
            for(int k = 0; k <4; ++k)
            {
                V[i*4+j] += B_t[i][k] * temp[k][j];
            }
        }
    }
    //float *fin = (float *)malloc(bs * p * q * ch * och * 4 * 4 * sizeof(float));
    float *fin = (float *)malloc(och*4*4*sizeof(float));
    uv<<<1,och>>>(tch, ch, fin, devU, V); 
    cudaDeviceSynchronize();
    // copy thrtile to devout for testing
    // int offset2 = (((tbs*p + tp)*q + tq)*ch + tch)*16;
    //LOOP(och)
    for(int toch = 0; toch<och; toch++)
    {
        for(int i = 0; i < 4; i++)
        {
            for(int j = 0; j < 4; j++)
            {
               devout[(((((tbs*och+toch)*p+tp)*q+tq)*ch+tch)*4 + i)*4 + j] = fin[(toch*4+i)*4+j];
               //cout << fin[(toch*4+i)*4+j] <<", ";
            }
         //out << endl;
        }
    }
    // sum along the channels, using log n summing
    // int k = ch, j = tch;
    // int offset3 = ((tbs*p + tp)*q + tq)*ch*16;
    for(int s = 1; s < ch; s *= 2)
    {
        if(tch % (2*s) == 0 && tch+s < ch)
        {
            LOOP(och)
            {
                for(int i = 0; i < 4; i++)
                {
                    for(int j = 0; j < 4; j++)
                    {
                        devout[(((((tbs*och+toch)*p+tp)*q+tq)*ch+tch)*4 + i)*4 + j] += devout[(((((tbs*och+toch)*p+tp)*q+tq)*ch+(tch+s))*4 + i)*4 + j];
                    }
                }
            }
        }
        __syncthreads();
    }
    if(tch == 0) // can do with tch == 0
    {
        // int offset = ((tbs*p + tp)*q + tq)*16;
        LOOP(och)
        {    
            for(int i = 0; i < 4; i++)
            {
                for(int j = 0; j < 4; j++)
                {
                    devsum[((((tbs*och+toch)*p+tp)*q+tq)*4 + i)*4 + j] = devout[(((((tbs*och+toch)*p+tp)*q+tq)*ch)*4 + i)*4 + j];
                }
            }
        }
    }
  if(tch == 0)
      amul<<<1,och>>>(tbs, tp, tq, bs, och, p, q, devsum, devY);
 cudaDeviceSynchronize();
}
void tilehost(int och, int ch, int bs, int h, int w, int pad, float *&in, int &p, int &q, int &outsize, float *&out, int &sumsize, float *&sum, int &ysize, float *&Y, float *kwt)
{
   // int p, q;
    float *devin, *devinnopad;
    int insize = bs * ch * h * w * sizeof(float);
    gpu_error(cudaMalloc((void **) & devinnopad, insize));
    gpu_error(cudaMemcpy(devinnopad, in, insize, cudaMemcpyHostToDevice));

    int newh, neww;
    newh = h + 2*pad;
    neww = w + 2*pad;
    if(newh%2)
        newh++;
    if(neww%2)
        neww++;
    if(newh < 4)
        newh = 4;
    if(neww < 4)
        neww = 4;

    insize = bs * ch * newh * neww * sizeof(float);
    gpu_error(cudaMalloc((void **) & devin, insize));

    // call padding
    dim3 padgrid(bs*ch, newh, neww);
    dim3 padblock(1, 1, 1);
 
    paddev<<<padgrid,padblock>>>(devin, devinnopad, h, w, pad);

    gpu_error(cudaFree(devinnopad));
    h = newh;
    w = neww;
    p = max((h-2)/2, 0);
    q = max((w-2)/2, 0);
    
    float *devout, *devsum, *devkwt, *devU, *devY;
    float *devfin;
    devout = devsum = nullptr;
    //int insize = bs * ch * h * w * sizeof(float);
    int kwtsize = och*ch*3*3*sizeof(float);
    int finsize = bs * p * q * ch * och * 4 * 4 * sizeof(float);
    outsize = bs * och * p * q * ch * 4 * 4 * sizeof(float);
    sumsize = bs * och * p * q * 4 * 4 * sizeof(float);
    ysize = bs * och * p * q * 2 * 2 * sizeof(float);
    int usize = och*ch*4*4*sizeof(float);
 
    //gpu_error(cudaMalloc((void **) & devin, insize));
    gpu_error(cudaMalloc((void **) & devout, outsize));
    gpu_error(cudaMalloc((void **) & devsum, sumsize));
    gpu_error(cudaMalloc((void **) & devkwt, kwtsize));
    gpu_error(cudaMalloc((void **) & devU, usize));
    gpu_error(cudaMalloc((void **) & devfin, finsize));
    gpu_error(cudaMalloc((void **) & devY, ysize));
    
    //gpu_error(cudaMemcpy(devin, in, insize, cudaMemcpyHostToDevice));
    gpu_error(cudaMemcpy(devkwt, kwt, kwtsize, cudaMemcpyHostToDevice));
    printf("mem done\n");
    // call the kernel function for tiling
    
    float *U = (float *)malloc(usize);
    precompute<<<och, ch>>>(och, ch, devkwt, devU);
    cudaSafeCall(cudaMemcpy(U, devU, usize, cudaMemcpyDeviceToHost));
    cout << och << endl;
    /*for(int toch=0; toch<och; toch++)
    {
        cout<<"{ ";
        LOOP(ch)
        {
            cout<<"{ ";
            for(int i = 0; i < 4; i++)
            {
                for(int j = 0; j < 4; j++)
                {
                    cout<<U[((toch*ch+tch)*4+i)*4+j]<<",";
                }
                cout<<";\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }*/
    
    dim3 grid(bs, p, q);  // 3-D
    dim3 block(ch, 1, 1); // 1-D
    tile<<<grid, block>>>(bs, p, q, ch, devin, devout, devsum, devY, devU, h, w, och, devfin);
    cudaSafeCall(cudaGetLastError());
    // copy from device to host to out.
    printf("tile done\n");
    delete in;
    out = (float *)malloc(outsize);
    sum = (float *)malloc(sumsize);
    Y = (float *)malloc(ysize);
    printf("lala\n");
    cudaSafeCall(cudaMemcpy(out, devout, outsize, cudaMemcpyDeviceToHost));
    cudaSafeCall(cudaMemcpy(sum, devsum, sumsize, cudaMemcpyDeviceToHost));
    cudaSafeCall(cudaMemcpy(Y, devY, ysize, cudaMemcpyDeviceToHost));
    printf("host mem cpy done\n");
    gpu_error(cudaFree(devin));
    gpu_error(cudaFree(devout));
    gpu_error(cudaFree(devsum));
    gpu_error(cudaFree(devkwt));
    gpu_error(cudaFree(devU));
    gpu_error(cudaFree(devfin));
    gpu_error(cudaFree(devY));
}
int main(void) 
{
    auto engine = default_random_engine(time(nullptr));
    auto rng = uniform_real_distribution<float>();
    int bs, ch, h, w, p, q, och, pad;
    
    bs = 2;
    ch = 2;
    h = 5;
    w = 5;
    och = 1;
    pad = 0;
    int insize = bs * ch * h * w * sizeof(float);
    int outsize, sumsize, ysize;
    float *in = new float[insize/sizeof(float)];
    float *t = in;
    float *out, *sum, *Y;
    float *kernel_weights = new float[och*3*3*ch];
    int tsize = och*ch*3*3;
    float *tkw = kernel_weights;
    LOOP(tsize)
    {
        tkw[ttsize] = 0;
    }
 tkw[0] = 1;
 tkw[9] = 1;
    LOOP(bs)
    {
        LOOP(ch)
        {
            LOOP(h)
            {
                LOOP(w)
                {
                    *(t++) = rng(engine);
                }
            }
        }
    }
    LOOP(bs)
    {
        cout<<"{ ";
        LOOP(ch)
        {
            cout<<"{ ";
            LOOP(h)
            {
                cout<<"{ ";
                LOOP(w)
                {
                    cout<<in[((tbs*ch+tch)*h+th)*w+tw]<<" ";
                }
                cout<<"}\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }
    cout<<"\nTiling and Summing\n";
    tilehost(och, ch, bs, h, w, pad, in, p, q, outsize, out, sumsize, sum, ysize, Y, kernel_weights);
    
    cout<<"\nTiling finished\n\n";
 LOOP(bs)
    {
        cout<<"{ ";   
    
    LOOP(och)
    {
        cout<<"{ ";
        LOOP(p)
        {
            cout<<"{ ";
            LOOP(q)
            {
                cout<<"{ ";
                LOOP(ch)
                {
                    cout<<"{ ";
                    for(int i = 0; i < 4; i++)
                    {
                        for(int j = 0; j < 4; j++)
                        {
                            cout<<out[(((((tbs*och+toch)*p+tp)*q+tq)*ch+tch)*4+i)*4+j]<<",";
                        }
                        cout<<";\n";
                    }
                    cout<<"}\n";
                }
                cout<<"}\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }
    }
    
    cout<<"\nSumming finished\n\n";
 /*
    LOOP(bs)
    {
        cout<<"{ ";
        LOOP(p)
        {
            cout<<"{ ";
            LOOP(q)
            {
                cout<<"{ ";
                for(int i = 0; i < 4; i++)
                {
                    for(int j = 0; j < 4; j++)
                    {
                        cout<<sum[(((tbs*p+tp)*q+tq)*4+i)*4+j]<<",";
                    }
                    cout<<";\n";
                }
                cout<<"}\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }*/
    LOOP(bs)
    {
        cout<<"{ ";
        LOOP(och)
        {
            cout<<"{ ";
            LOOP(p)
            {
                cout<<"{ ";
                LOOP(q)
                {
                    cout<<"{ ";
                    for(int i = 0; i < 2; i++)
                    {
                        for(int j = 0; j < 2; j++)
                        {
                            cout<<Y[((((tbs*och+toch)*p+tp)*q+tq)*2+i)*2+j]<<",";
                        }
                        cout<<";\n";
                    }
                    cout<<"}\n";
                }
                cout<<"}\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }
    return 0;
}

'File written in /content/src/helloCUDA.cu'

In [0]:
!nvcc -std=c++11 -arch=sm_35 -rdc=true -Xcompiler -rdynamic -lineinfo /content/src/helloCUDA.cu -o /content/src/helloCUDA

In [28]:
!cuda-memcheck /content/src/helloCUDA

========= CUDA-MEMCHECK
{ { { 0.828405 0.00310428 0.173661 0.721651 0.786249 }
{ 0.487554 0.314881 0.205982 0.947265 0.675343 }
{ 0.484091 0.113343 0.948366 0.186256 0.397531 }
{ 0.310188 0.323091 0.190648 0.227116 0.142369 }
{ 0.793699 0.69114 0.99521 0.489044 0.358081 }
}
{ { 0.265835 0.881885 0.841328 0.195957 0.446755 }
{ 0.616515 0.771662 0.322451 0.438571 0.0622449 }
{ 0.149394 0.858909 0.675668 0.953134 0.325084 }
{ 0.6807 0.522493 0.544733 0.332716 0.959768 }
{ 0.818202 0.517626 0.733974 0.894942 0.293692 }
}
}
{ { { 0.086177 0.376164 0.188265 0.171837 0.05803 }
{ 0.310758 0.902457 0.601807 0.563008 0.482741 }
{ 0.43355 0.669777 0.946832 0.398385 0.648675 }
{ 0.284766 0.0557035 0.20891 0.146084 0.226031 }
{ 0.909347 0.392501 0.77247 0.910719 0.45095 }
}
{ { 0.112325 0.846841 0.860779 0.118318 0.56852 }
{ 0.108346 0.969256 0.283512 0.987704 0.348562 }
{ 0.289134 0.469653 0.463776 0.684845 0.193967 }
{ 0.00776305 0.473668 0.93842 0.0168944 0.9434 }
{ 0.728984 0.0398879 0.396429 0

In [29]:
!/content/src/helloCUDA #< input.txt

{ { { 0.828483 0.318482 0.729714 0.308152 0.113921 }
{ 0.677146 0.785327 0.99463 0.740229 0.0222716 }
{ 0.319112 0.307506 0.258019 0.531977 0.932148 }
{ 0.60719 0.0345849 0.26763 0.0612727 0.810791 }
{ 0.968558 0.558867 0.884976 0.793409 0.81975 }
}
{ { 0.535122 0.801534 0.380517 0.35595 0.458661 }
{ 0.719724 0.393642 0.946981 0.913216 0.423064 }
{ 0.431736 0.185294 0.239773 0.863508 0.97847 }
{ 0.151089 0.347996 0.771337 0.866274 0.474954 }
{ 0.545061 0.833973 0.581045 0.622042 0.657806 }
}
}
{ { { 0.751125 0.153482 0.566009 0.920905 0.65068 }
{ 0.97583 0.767631 0.578369 0.655089 0.0883846 }
{ 0.480081 0.715006 0.109751 0.588714 0.515096 }
{ 0.224532 0.717149 0.116239 0.623726 0.963522 }
{ 0.916418 0.233923 0.543288 0.0488932 0.748424 }
}
{ { 0.756783 0.256641 0.361619 0.73346 0.270565 }
{ 0.386519 0.22291 0.442189 0.872257 0.0310338 }
{ 0.584568 0.831899 0.717896 0.679424 0.0796894 }
{ 0.339712 0.536886 0.448572 0.149631 0.854536 }
{ 0.1888 0.160696 0.811535 0.469485 0.631726 }
}
}

